In [1]:
# set paths

RESULTS_DIR = "D:/Igor/Research_USF/University of South Florida/Mao, Wenbin - Igor/Febio-Models/Active-Models/PAQ/Myo_test"

# define headers
HEADER_KEY = ['fileName']
HEADER_TIM = ['time']
HEADER_STR = ['sx','sy','sz','sxy','sxz','syz']
HEADER_DIS = ['ux','uy','uz']
HEADER_POS = ['x','y','z']
HEADER_FAL = ['fail']

# define last simulation timestamp
LAST_TIMESTAMP = 0.2

Load files

In [2]:
# define functions to find files
from os import listdir, makedirs
from os.path import isfile, join, isdir

def find_files(path_to_folder, condition=None):
	if (condition):
		if (condition[0] == "fileFormat"):
			_l = len(condition[1])
			return [(join(path_to_folder,f), f,f[:-_l - 1]) for f in listdir(path_to_folder) if isfile(join(path_to_folder, f)) and f[-_l:] == condition[1]]
		else:
			raise(ValueError("Condition should be a tuple where the first argument is the condition name and the second is its value"))
	else:
		return [(join(path_to_folder,f), f, f.rsplit(".")[0]) for f in listdir(path_to_folder) if isfile(join(path_to_folder, f))]


def find_folders(path_to_folder, condition=None):
    return [(join(path_to_folder,f), f, f.rsplit(".")[0]) for f in listdir(path_to_folder) if isdir(join(path_to_folder, f))]


In [3]:
# find files

directories = find_folders(RESULTS_DIR)
dir_with_files = []
for dir in directories:
    dir_with_files.append( (dir[-1], find_files(dir[0], ("fileFormat", "txt"))) )

dis_files = {}
str_files = {}
pos_files = {}
for key, files in dir_with_files:
    for (fp, ff, fn) in files:
        fs = fn.split("_")
        if fs[0] == 'displacement':
            dis_files[key] = (fp, ff, fn)
        elif fs[0] == 'stress':
            str_files[key] = (fp, ff, fn)
        elif fs[0] == 'position':
            pos_files[key] = (fp, ff, fn)
files = [str_files, dis_files, pos_files]


In [4]:
files

[{'myo_hex_8_coarse_PAQ': ('D:/Igor/Research_USF/University of South Florida/Mao, Wenbin - Igor/Febio-Models/Active-Models/PAQ/Myo_test\\myo_hex_8_coarse_PAQ\\stress.txt',
   'stress.txt',
   'stress')},
 {'myo_hex_8_coarse_PAQ': ('D:/Igor/Research_USF/University of South Florida/Mao, Wenbin - Igor/Febio-Models/Active-Models/PAQ/Myo_test\\myo_hex_8_coarse_PAQ\\displacement_node_out.txt',
   'displacement_node_out.txt',
   'displacement_node_out')},
 {'myo_hex_8_coarse_PAQ': ('D:/Igor/Research_USF/University of South Florida/Mao, Wenbin - Igor/Febio-Models/Active-Models/PAQ/Myo_test\\myo_hex_8_coarse_PAQ\\position_node_out.txt',
   'position_node_out.txt',
   'position_node_out')}]

Decode data

In [5]:
# set function to read data based on timesteps of the log file
import re
import numpy as np

def decode_data(file):
	data = {}
	with open(file, 'r') as datafile:
		for line in datafile:
			if line.find("*Time") != -1:
				time = float(line.split("=")[1])
				data[time] = []
			elif line.find("*") == -1:
				line = re.sub("\n", '', line)
				line = re.sub(",", '', line)
				str_data = line.split(" ")
				node = int(str_data[0])
				node_data = [float(s) for s in str_data[1:]]
				data[time].extend(node_data)
	for key in data:
		data[key] = np.array(data[key])
	return data

In [6]:
def create_header():
    a = HEADER_KEY + HEADER_TIM + HEADER_FAL + HEADER_POS + HEADER_DIS + HEADER_STR
    return a

In [7]:
# str_data = decode_data(str_files['with_load_myo_hex_coarse_1_epi_60_endo_-60'][0])
# str_data.keys()


In [8]:
import pandas as pd

def create_nodal_df(data, header, only_include_first_and_last=True):
    """ creates a dictionary with keys representing timesteps """
    dfs = {}
    datakeys = data.keys()
    if only_include_first_and_last:
        datakeys = [datakeys[0], datakeys[-1]]
    for key in datakeys:
        _subdata = np.array_split(data[key], len(data[key])/len(header))
        dfs[key] = pd.DataFrame(_subdata, columns=header)
    return dfs


In [9]:
def create_df(data, header, ix_type, only_include_first_and_last=True):
    _datastacked = np.empty([1, len(header) + 2])

    datakeys = list(data.keys())
    datakeys.sort()
    if only_include_first_and_last:
        if len(datakeys) > 1:
            datakeys = [datakeys[0], datakeys[-1]]
    for key in datakeys:
        _subdata = np.array(np.array_split(data[key], len(data[key])/len(header)))
        _timearr = np.ones((len(_subdata), 1)) * key
        _nodearr = np.arange(1,len(_subdata)+1,dtype=int).reshape((len(_subdata),1))
        _subdata = np.hstack((_timearr,_nodearr, _subdata))
        _datastacked = np.concatenate((_datastacked, _subdata))

    new_header = np.hstack((['time', ix_type], header))
    df = pd.DataFrame(_datastacked, columns=new_header)
    df.drop([0], inplace=True)
    df = df.reset_index(drop=True)
    return df


In [10]:
# str_data

In [11]:
# str_dfs = create_df(str_data, HEADER_STR, "elem")


In [12]:
# str_dfs

In [13]:
import pandas as pd
import numpy as np

header = create_header()

dfs = []
elem_df = pd.DataFrame()
node_df = pd.DataFrame()

tpm_dir = "./tmp"
tpm_elem_dir = join(tpm_dir, "elems")
tpm_node_dir = join(tpm_dir, "nodes")

for d in [tpm_dir, tpm_elem_dir, tpm_node_dir]:
    if not os.path.isdir(d):
        os.makedirs(d)

max_count = len(str_files)

print("Progress: 0 %")
for i, key in enumerate(str_files):
    df = pd.DataFrame(columns=header)

    str_data = decode_data(str_files[key][0])
    dis_data = decode_data(dis_files[key][0])
    pos_data = decode_data(pos_files[key][0])

    str_df = create_df(str_data, HEADER_STR, "elem")
    dis_df = create_df(dis_data, HEADER_DIS, "node")
    pos_df = create_df(pos_data, HEADER_POS, "node")

    

    l_elem_df = str_df
    # l_node_df = pd.concat((pos_df, dis_df), axis=1)
    l_node_df = pos_df.merge(dis_df, how='outer')

    l_elem_df["fileName"] = key
    l_node_df["fileName"] = key

    l_elem_df.to_pickle(join(tpm_elem_dir, "elem_df_%s.pickle" % key))
    l_node_df.to_pickle(join(tpm_node_dir, "node_df_%s.pickle" % key))

    print("Progress: {} %", 100 * ((i+1)/max_count))


Progress: 0 %
Progress: {} % 100.0


In [14]:
# l_node_df

In [15]:
def compile_data(files):
	frames = []
	total_files = len(files)
	for i, (fp, _, _) in enumerate(files):
		ndf = pd.read_pickle(fp)
		frames.append(ndf)
		print("compiled pickle: ", (i + 1) /total_files)
		df = pd.concat(frames, ignore_index=True)
	return df

elem_pickle_files = find_files(tpm_elem_dir, ("fileFormat","pickle"))
node_pickle_files = find_files(tpm_node_dir, ("fileFormat","pickle"))

elem_df = compile_data(elem_pickle_files)
node_df = compile_data(node_pickle_files)

out_dir = "./compiled_data"
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

elem_df.to_pickle(join(out_dir, "elems_compiled" + ".pickle"))
node_df.to_pickle(join(out_dir, "nodes_compiled" + ".pickle"))


compiled pickle:  1.0
compiled pickle:  1.0


In [16]:
node_df

,time,node,x,y,z,ux,uy,uz,fileName
0,0.004,1.0,-9.344340e-03,-1.942380e-02,-65.1808,-0.009344,-0.019424,-0.180786,myo_hex_8_coarse_PAQ
1,0.004,2.0,2.143130e-15,-5.249160e-31,-75.5475,0.000000,0.000000,-0.547491,myo_hex_8_coarse_PAQ
2,0.004,3.0,3.414030e+01,-1.449470e+00,20.0000,0.407643,-1.449470,0.000000,myo_hex_8_coarse_PAQ
3,0.004,4.0,2.444760e+01,1.123150e+00,20.0000,0.660492,1.123150,0.000000,myo_hex_8_coarse_PAQ
4,0.004,5.0,-2.344110e-03,-9.428740e-03,-66.5087,-0.002344,-0.009429,-0.258713,myo_hex_8_coarse_PAQ
...,...,...,...,...,...,...,...,...,...
45013,0.200,22505.0,2.733940e+01,-2.168970e+01,17.5604,0.845670,-2.440830,-0.306306,myo_hex_8_coarse_PAQ
45014,0.200,22506.0,2.984460e+01,-1.809090e+01,17.5538,1.147230,-2.314400,-0.312968,myo_hex_8_coarse_PAQ
45015,0.200,22507.0,3.187140e+01,-1.420330e+01,17.5553,1.423050,-2.147970,-0.311459,myo_hex_8_coarse_PAQ
45016,0.200,22508.0,3.339640e+01,-1.009450e+01,17.5532,1.677140,-1.950340,-0.313567,myo_hex_8_coarse_PAQ
